## Importing subtitles data

In [68]:
import glob
import unicodedata
import codecs

In [102]:
BASE_PATH = './friends-season-1/'
idcs = [str(_).zfill(2) for _ in range(1, 25)]
subtitles_data = []

for idx in idcs[:]:
    fn = glob.glob(BASE_PATH + '*1x' + idx + '*')
    f = open(fn[0], 'rb')
    
    content = f.read().decode('utf8', 'ignore')
    subtitles_data.append(content)

## Extracting only lines (대사) from the files

In [99]:
def extract_dialogue(file_content):
    dialogue = []
    lines = file_content.split('\r\n\r\n')
    for line in lines:
        dialogue.extend(line.split('\r\n')[2:])
    return dialogue

In [104]:
subtitles_lines = []
for content in subtitles_data:
    subtitles_lines.append(extract_dialogue(content))

## Recognizing Proper Noun 

In [117]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

In [155]:
pps = []
for line in subtitles_lines[0][3:-1]:
#     tagged_sent = pos_tag([token.lower() for token in word_tokenize(line)])
    tagged_sent = pos_tag(word_tokenize(line))
    pps.extend ([word for word, pos in tagged_sent if pos == 'NNP'])